# Importo le librerie necessarie

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import re
import pandas_profiling
WHITE_SPACE_PATTERN = re.compile(r' +')

In [3]:
from platform import python_version
print(python_version())
print(pd.__version__)
print(np.__version__)

3.7.4
1.3.5
1.21.5


# Effettuo scraping del calendario delle gare in cui partecipa l'Italia

In [4]:
def getSoup(data):
    base_url = "https://tokyo2020.coni.it/it/azzurri-in-gara/ingaradettaglio.html?giorno={}&tmpl=component".format(data)
    print(base_url)
    response = requests.get(base_url)
    html_page = response.text
    soup = bs(html_page)
    return soup

In [5]:
def getTable(soup):
    table = soup.find_all('table')[0]
    
    #get headings
    headings = []
    for item in table.find_all("thead")[0].find_all("tr")[0].find_all("th"):
        item = (item.text).strip()
        headings.append(item)
    
    #get body
    body = []
    length = len(table.find_all("tbody")[0].find_all("tr"))

    for i in range(length):
        row = []
        for item in table.find_all("tbody")[0].find_all("tr")[i].find_all("td"):
            item = (item.text).strip()
            row.append(item)
        body.append(row)
    
    df = pd.DataFrame(data=body,columns=headings)
    return df

In [6]:
#le olimpiadi iniziano il 23 luglio e terminano l'8 agosto
sdate = date(2021,7,23)   
edate = date(2021,8,9)
date_list = pd.date_range(sdate,edate-timedelta(days=1),freq='d').strftime('')

In [7]:
temp = pd.DataFrame(columns=[])

for data in date_list:
    soup = getSoup(data)
    df = getTable(soup)
    df["inizio"] = "{} ".format(data) + df["ora ita"].str[:5]
    df["fine"] = "{} ".format(data) + df["ora ita"].str[-5:]
    temp = pd.concat([temp, df]) 
    
temp = temp.reset_index(drop=True)

https://tokyo2020.coni.it/it/azzurri-in-gara/ingaradettaglio.html?giorno=2021-07-23&tmpl=component
https://tokyo2020.coni.it/it/azzurri-in-gara/ingaradettaglio.html?giorno=2021-07-24&tmpl=component
https://tokyo2020.coni.it/it/azzurri-in-gara/ingaradettaglio.html?giorno=2021-07-25&tmpl=component
https://tokyo2020.coni.it/it/azzurri-in-gara/ingaradettaglio.html?giorno=2021-07-26&tmpl=component
https://tokyo2020.coni.it/it/azzurri-in-gara/ingaradettaglio.html?giorno=2021-07-27&tmpl=component
https://tokyo2020.coni.it/it/azzurri-in-gara/ingaradettaglio.html?giorno=2021-07-28&tmpl=component
https://tokyo2020.coni.it/it/azzurri-in-gara/ingaradettaglio.html?giorno=2021-07-29&tmpl=component
https://tokyo2020.coni.it/it/azzurri-in-gara/ingaradettaglio.html?giorno=2021-07-30&tmpl=component
https://tokyo2020.coni.it/it/azzurri-in-gara/ingaradettaglio.html?giorno=2021-07-31&tmpl=component
https://tokyo2020.coni.it/it/azzurri-in-gara/ingaradettaglio.html?giorno=2021-08-01&tmpl=component
https://to

In [8]:
temp

,ora jap,ora ita,sport,evento,impianto,Azzurri in gara,inizio,fine
0,08:50 -\n ...,01:50 -\n ...,Canottaggio,Batterie Singolo U,Sea Forest Waterway,GENNARO DI MAURO,2021-07-23 01:50,2021-07-23 02:25
1,09:00 -\n ...,02:00 -\n ...,Tiro con l'arco,Torneo di qualificazione Ind. D,Yumenoshima Park Archery Field,TATIANA ANDREOLI\n\n\nLUCILLA BOARI \n\n\nCHIA...,2021-07-23 02:00,2021-07-23 04:00
2,11:00 -\n ...,04:00 -\n ...,Canottaggio,Batterie Due di coppia D,Sea Forest Waterway,CHIARA ONDOLI\n\n\nALESSANDRA PATELLI,2021-07-23 04:00,2021-07-23 04:25
3,11:30 -\n ...,04:30 -\n ...,Canottaggio,Batterie Quattro di coppia U,Sea Forest Waterway,GIACOMO GENTILI\n\n\nANDREA PANIZZA\n\n\nLUCA ...,2021-07-23 04:30,2021-07-23 04:45
4,11:50 -\n ...,04:50 -\n ...,Canottaggio,Batterie Quattro di coppia D,Sea Forest Waterway,STEFANIA GOBBI\n\n\nVALENTINA ISEPPI\n\n\nVERO...,2021-07-23 04:50,2021-07-23 05:10
...,...,...,...,...,...,...,...,...
497,10:00 -\n ...,03:00 -\n ...,Ciclismo,Pista - Omnuim Scratch D,Izu Velodrome,ELISA BALSAMO,2021-08-08 03:00,2021-08-08 06:15
498,10:45 -\n ...,03:45 -\n ...,Ciclismo,Pista - Omnium gara a tempo D,Izu Velodrome,ELISA BALSAMO,2021-08-08 03:45,2021-08-08 06:15
499,11:00 -\n ...,04:00 -\n ...,Ginnastica,Ritmica - Finale All-Around Gruppi,Ariake Gymnastics Centre,MARTINA CENTOFANTI\n\n\nAGNESE DURANTI\n\n\nAL...,2021-08-08 04:00,2021-08-08 05:55
500,11:26 -\n ...,04:26 -\n ...,Ciclismo,Pista - Omnium gara a eliminazione D,Izu Velodrome,ELISA BALSAMO,2021-08-08 04:26,2021-08-08 06:15


# Risolvo problemi di qualità dei dati

I problemi riscontrati sono i seguenti:
- l'evento della riga 233 presenta stEffetta, invece di stAffetta;
- ci sono 53 celle vuote nella colonna azzurri in gara, di cui 28 sono stati inseriti manualmente, mentre i restanti 25 riguardano gare alle quali non hanno partecipato italiani e vengono quindi rimosse le rispettive righe;
- accenti e spazi bianchi in azzurri in gara;
- ci sono 2 eventi con orario di fine e inizio invertiti;
- ci sono 11 eventi con giorno di inizio errato.

In [9]:
temp.evento.loc[233] = 'Batterie Staffetta 4x200m stile libero D'

In [10]:
np.where(temp.applymap(lambda x: x == '')) #53

(array([104, 127, 128, 167, 177, 191, 192, 210, 216, 217, 218, 219, 220,
        221, 222, 223, 225, 228, 233, 267, 277, 292, 302, 303, 321, 324,
        330, 367, 368, 369, 372, 373, 379, 389, 392, 393, 395, 398, 404,
        406, 407, 408, 416, 424, 425, 426, 462, 469, 476, 480, 490, 492,
        493]),
 array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5]))

In [11]:
# aggiungo eventi manualmente

temp.at[104,"Azzurri in gara"] = "LORENZO ZAZZERI, ALESSANDRO MIRESSI, MANUEL FRIGO, SANTO CONDORELLI"
temp.at[127,"Azzurri in gara"] = "LORENZO ZAZZERI, ALESSANDRO MIRESSI, THOMAS CECCON, MANUEL FRIGO"
temp.at[128,"Azzurri in gara"] = "ASIA LANZI"
temp.at[167,"Azzurri in gara"] = "MARCO SUPPINI, SOFIA CECCARELLO"
temp.at[177,"Azzurri in gara"] = "MARCO DE TULLIO, STEFANO DI COLA, MATTEO CIAMPI, FILIPPO MEGLI"
temp.at[192,"Azzurri in gara"] = "ALICE BELLANDI"
temp.at[222,"Azzurri in gara"] = "ALICE BELLANDI"
temp.at[210,"Azzurri in gara"] = "LUCA CURATOLI, ALDO MONTANO, ENRICO BERRÈ"
temp.at[228,"Azzurri in gara"] = "LUCA CURATOLI, ALDO MONTANO, ENRICO BERRÈ"
temp.at[233,"Azzurri in gara"] = "FEDERICA PELLEGRINI, STEFANIA PIROZZI, ANNA CHIARA MASCOLO, GIULIA VETRANO"
temp.at[267,"Azzurri in gara"] = "FEDERICA PELLEGRINI, ELENA DI LIDDO, SIMONE SABBIONI, NICOLÒ MARTINENGHI"
temp.at[292,"Azzurri in gara"] = "GIOVANNI DE GENNARO"
temp.at[302,"Azzurri in gara"] = "FEDERICA PELLEGRINI, ELENA DI LIDDO, MARGHERITA PANZIERA, ARIANNA CASTIGLIONI"
temp.at[303,"Azzurri in gara"] = "FEDERICO BURDISSO, ALESSANDRO MIRESSI, THOMAS CECCON, NICOLÒ MARTINENGHI"
temp.at[367,"Azzurri in gara"] = "FRANCESCA GENZO"
temp.at[368,"Azzurri in gara"] = "SAMUELE BURGO"
temp.at[372,"Azzurri in gara"] = "LETIZIA PATERNOSTER, ELISA BALSAMO, RACHELE BARBIERI, VITTORIA GUAZZINI"
temp.at[379,"Azzurri in gara"] = "GLORIA HOOPER, DALIA KADDARI"
temp.at[393,"Azzurri in gara"] = "FRANCESCA GENZO"
temp.at[395,"Azzurri in gara"] = "SAMUELE BURGO"
temp.at[398,"Azzurri in gara"] = "SAMUELE BURGO"
temp.at[416,"Azzurri in gara"] = "ESEOSA FOSTINE DESALU'"
temp.at[425,"Azzurri in gara"] = "FRANCESCA GENZO"
temp.at[426,"Azzurri in gara"] = "LUCA BECCARO, SAMUELE BURGO"
temp.at[462,"Azzurri in gara"] = "MARIA BENEDICTA CHIGBOLU, ALICE MANGIONE, REBECCA BORGA, PETRA NARDELLI"
temp.at[469,"Azzurri in gara"] = "ABRAHAM CONYEDO RUANO"
temp.at[480,"Azzurri in gara"] = "LUIGI BUSÀ"
temp.at[492,"Azzurri in gara"] = "ABRAHAM CONYEDO RUANO"

#rimuovo eventi dove non hanno partecipato italiani
temp = temp.drop([191, 216, 217, 218, 219, 220, 221, 223, 225, 277, 321, 324, 330, 369, 373, 389, 392, 404, 406, 407, 408, 424, 476, 490, 493])
temp = temp.reset_index(drop=True)

In [13]:
#elimino colonne
temp = temp.drop(columns=['ora jap', 'ora ita'])

#sistemo accenti e spazi bianchi
temp["Azzurri in gara"] = temp["Azzurri in gara"].str.replace('\n\n\n', ', ')
temp["Azzurri in gara"] = temp["Azzurri in gara"].str.replace(WHITE_SPACE_PATTERN, ' ')
temp["Azzurri in gara"] = temp["Azzurri in gara"].str.replace("A'", 'À')
temp["Azzurri in gara"] = temp["Azzurri in gara"].str.replace("E'", 'È')
temp["Azzurri in gara"] = temp["Azzurri in gara"].str.replace("I'", 'Ì')
temp["Azzurri in gara"] = temp["Azzurri in gara"].str.replace("O'", 'Ò')
temp["Azzurri in gara"] = temp["Azzurri in gara"].str.replace("U'", 'Ù')

temp["Azzurri in gara"] = temp["Azzurri in gara"].str.split(pat=", ")
temp = temp.explode("Azzurri in gara")
temp["Azzurri in gara"] = temp["Azzurri in gara"].str.strip()

temp = temp.reset_index(drop=True)
temp["Azzurri in gara"].loc[991] = temp["Azzurri in gara"].loc[991].replace("ESEOSA FOSTINE DESALÙ", 'ESEOSA FOSTINE DESALU')

temp

,sport,evento,impianto,Azzurri in gara,inizio,fine
0,Canottaggio,Batterie Singolo U,Sea Forest Waterway,GENNARO DI MAURO,2021-07-23 01:50,2021-07-23 02:25
1,Tiro con l'arco,Torneo di qualificazione Ind. D,Yumenoshima Park Archery Field,TATIANA ANDREOLI,2021-07-23 02:00,2021-07-23 04:00
2,Tiro con l'arco,Torneo di qualificazione Ind. D,Yumenoshima Park Archery Field,LUCILLA BOARI,2021-07-23 02:00,2021-07-23 04:00
3,Tiro con l'arco,Torneo di qualificazione Ind. D,Yumenoshima Park Archery Field,CHIARA REBAGLIATI,2021-07-23 02:00,2021-07-23 04:00
4,Canottaggio,Batterie Due di coppia D,Sea Forest Waterway,CHIARA ONDOLI,2021-07-23 04:00,2021-07-23 04:25
...,...,...,...,...,...,...
1185,Ginnastica,Ritmica - Finale All-Around Gruppi,Ariake Gymnastics Centre,ALESSIA MAURELLI,2021-08-08 04:00,2021-08-08 05:55
1186,Ginnastica,Ritmica - Finale All-Around Gruppi,Ariake Gymnastics Centre,DANIELA MOGUREAN,2021-08-08 04:00,2021-08-08 05:55
1187,Ginnastica,Ritmica - Finale All-Around Gruppi,Ariake Gymnastics Centre,MARTINA SANTANDREA,2021-08-08 04:00,2021-08-08 05:55
1188,Ciclismo,Pista - Omnium gara a eliminazione D,Izu Velodrome,ELISA BALSAMO,2021-08-08 04:26,2021-08-08 06:15


In [14]:
temp['durata']= pd.to_datetime(temp.fine)-pd.to_datetime(temp.inizio)
temp

,sport,evento,impianto,Azzurri in gara,inizio,fine,durata
0,Canottaggio,Batterie Singolo U,Sea Forest Waterway,GENNARO DI MAURO,2021-07-23 01:50,2021-07-23 02:25,0 days 00:35:00
1,Tiro con l'arco,Torneo di qualificazione Ind. D,Yumenoshima Park Archery Field,TATIANA ANDREOLI,2021-07-23 02:00,2021-07-23 04:00,0 days 02:00:00
2,Tiro con l'arco,Torneo di qualificazione Ind. D,Yumenoshima Park Archery Field,LUCILLA BOARI,2021-07-23 02:00,2021-07-23 04:00,0 days 02:00:00
3,Tiro con l'arco,Torneo di qualificazione Ind. D,Yumenoshima Park Archery Field,CHIARA REBAGLIATI,2021-07-23 02:00,2021-07-23 04:00,0 days 02:00:00
4,Canottaggio,Batterie Due di coppia D,Sea Forest Waterway,CHIARA ONDOLI,2021-07-23 04:00,2021-07-23 04:25,0 days 00:25:00
...,...,...,...,...,...,...,...
1185,Ginnastica,Ritmica - Finale All-Around Gruppi,Ariake Gymnastics Centre,ALESSIA MAURELLI,2021-08-08 04:00,2021-08-08 05:55,0 days 01:55:00
1186,Ginnastica,Ritmica - Finale All-Around Gruppi,Ariake Gymnastics Centre,DANIELA MOGUREAN,2021-08-08 04:00,2021-08-08 05:55,0 days 01:55:00
1187,Ginnastica,Ritmica - Finale All-Around Gruppi,Ariake Gymnastics Centre,MARTINA SANTANDREA,2021-08-08 04:00,2021-08-08 05:55,0 days 01:55:00
1188,Ciclismo,Pista - Omnium gara a eliminazione D,Izu Velodrome,ELISA BALSAMO,2021-08-08 04:26,2021-08-08 06:15,0 days 01:49:00


In [15]:
errati= 0
for i in range(1190):
    if temp.durata.loc[i].days == -1:
        print(temp.loc[i])
        errati += 1
        
print('numero totale righe con orario o giorno errati: ' + str(errati))

sport                       Canottaggio
evento             Batterie Due senza U
impianto            Sea Forest Waterway
Azzurri in gara       GIOVANNI ABAGNALE
inizio                 2021-07-24 02:50
fine                   2021-07-24 02:20
durata                -1 days +23:30:00
Name: 38, dtype: object
sport                       Canottaggio
evento             Batterie Due senza U
impianto            Sea Forest Waterway
Azzurri in gara       MARCO DI COSTANZO
inizio                 2021-07-24 02:50
fine                   2021-07-24 02:20
durata                -1 days +23:30:00
Name: 39, dtype: object
sport                       Triathlon
evento                         Gara U
impianto           Odaiba Marine Park
Azzurri in gara     GIANLUCA POZZATTI
inizio               2021-07-26 23:30
fine                 2021-07-26 02:00
durata              -1 days +02:30:00
Name: 272, dtype: object
sport                         Triathlon
evento                           Gara U
impianto             

In [17]:
temp.inizio.loc[38]= temp.inizio.loc[38].replace('2021-07-24 02:50', '2021-07-24 02:20')
temp.inizio.loc[39]= temp.inizio.loc[39].replace('2021-07-24 02:50', '2021-07-24 02:20')
temp.fine.loc[38]= temp.fine.loc[38].replace('2021-07-24 02:20', '2021-07-24 02:50')
temp.fine.loc[39]= temp.fine.loc[39].replace('2021-07-24 02:20', '2021-07-24 02:50')


temp.inizio.loc[272]= temp.inizio.loc[272].replace('2021-07-26 23:30', '2021-07-25 23:30')
temp.inizio.loc[273]= temp.inizio.loc[273].replace('2021-07-26 23:30', '2021-07-25 23:30')

temp.inizio.loc[365]= temp.inizio.loc[365].replace('2021-07-27 23:30', '2021-07-26 23:30')
temp.inizio.loc[366]= temp.inizio.loc[366].replace('2021-07-27 23:30', '2021-07-26 23:30')
temp.inizio.loc[367]= temp.inizio.loc[367].replace('2021-07-27 23:30', '2021-07-26 23:30')

temp.inizio.loc[994]= temp.inizio.loc[994].replace('2021-08-04 23:30', '2021-08-03 23:30')

temp.inizio.loc[1046]= temp.inizio.loc[1046].replace('2021-08-05 23:30', '2021-08-04 23:30')
temp.inizio.loc[1047]= temp.inizio.loc[1047].replace('2021-08-05 23:30', '2021-08-04 23:30')

temp.inizio.loc[1091]= temp.inizio.loc[1091].replace('2021-08-06 22:30', '2021-08-05 22:30')
temp.inizio.loc[1092]= temp.inizio.loc[1092].replace('2021-08-06 22:30', '2021-08-05 22:30')
temp.inizio.loc[1093]= temp.inizio.loc[1093].replace('2021-08-06 22:30', '2021-08-05 22:30')

In [18]:
temp['durata']= pd.to_datetime(temp.fine)-pd.to_datetime(temp.inizio)
temp

,sport,evento,impianto,Azzurri in gara,inizio,fine,durata
0,Canottaggio,Batterie Singolo U,Sea Forest Waterway,GENNARO DI MAURO,2021-07-23 01:50,2021-07-23 02:25,0 days 00:35:00
1,Tiro con l'arco,Torneo di qualificazione Ind. D,Yumenoshima Park Archery Field,TATIANA ANDREOLI,2021-07-23 02:00,2021-07-23 04:00,0 days 02:00:00
2,Tiro con l'arco,Torneo di qualificazione Ind. D,Yumenoshima Park Archery Field,LUCILLA BOARI,2021-07-23 02:00,2021-07-23 04:00,0 days 02:00:00
3,Tiro con l'arco,Torneo di qualificazione Ind. D,Yumenoshima Park Archery Field,CHIARA REBAGLIATI,2021-07-23 02:00,2021-07-23 04:00,0 days 02:00:00
4,Canottaggio,Batterie Due di coppia D,Sea Forest Waterway,CHIARA ONDOLI,2021-07-23 04:00,2021-07-23 04:25,0 days 00:25:00
...,...,...,...,...,...,...,...
1185,Ginnastica,Ritmica - Finale All-Around Gruppi,Ariake Gymnastics Centre,ALESSIA MAURELLI,2021-08-08 04:00,2021-08-08 05:55,0 days 01:55:00
1186,Ginnastica,Ritmica - Finale All-Around Gruppi,Ariake Gymnastics Centre,DANIELA MOGUREAN,2021-08-08 04:00,2021-08-08 05:55,0 days 01:55:00
1187,Ginnastica,Ritmica - Finale All-Around Gruppi,Ariake Gymnastics Centre,MARTINA SANTANDREA,2021-08-08 04:00,2021-08-08 05:55,0 days 01:55:00
1188,Ciclismo,Pista - Omnium gara a eliminazione D,Izu Velodrome,ELISA BALSAMO,2021-08-08 04:26,2021-08-08 06:15,0 days 01:49:00


In [19]:
errati= 0
for i in range(1190):
    if temp.durata.loc[i].days == -1:
        print(temp.loc[i])
        errati += 1
        
print('numero totale righe con orario o giorno errati: ' + str(errati))

numero totale righe con orario o giorno errati: 0


In [20]:
temp = temp.drop(columns=['durata'])

In [21]:
temp['Azzurri in gara'].unique()

array(['GENNARO DI MAURO', 'TATIANA ANDREOLI', 'LUCILLA BOARI',
       'CHIARA REBAGLIATI', 'CHIARA ONDOLI', 'ALESSANDRA PATELLI',
       'GIACOMO GENTILI', 'ANDREA PANIZZA', 'LUCA RAMBALDI',
       'SIMONE VENIER', 'STEFANIA GOBBI', 'VALENTINA ISEPPI',
       'VERONICA LISI', 'ALESSANDRA MONTESANO', 'MAURO NESPOLI',
       'SOFIA CECCARELLO', 'ROSSELLA FIAMINGO', 'FEDERICA ISOLA',
       'MARA NAVARRIA', 'SIMONE ANZANI', 'MASSIMO COLACI',
       'GIANLUCA GALASSI', 'SIMONE GIANNELLI', 'OSMANY JUANTORENA',
       'JIRI KOVAR', 'DANIELE LAVIA', 'ALESSANDRO MICHIELETTO',
       'MATTEO PIANO', 'RICCARDO SBERTOLI', 'LUCA VETTORI',
       'IVAN ZAYTSEV', 'AISHA ROCEK', 'KIRI TONTODONATI', 'ENRICO BERRÈ',
       'LUCA CURATOLI', 'LUIGI SAMELE', 'GIOVANNI ABAGNALE',
       'MARCO DI COSTANZO', 'FEDERICA CESARINI', 'VALENTINA RODINI',
       "VITO DELL'AQUILA", 'STEFANO OPPO', 'PIETRO WILLY RUTA',
       'ALBERTO BETTIOL', 'DAMIANO CARUSO', 'GIULIO CICCONE',
       'GIANNI MOSCON', 'VINCENZO 

In [24]:
temp.to_csv('CalendarioAzzurri.csv')

# Effettuo scraping della lista di atleti italiani

In [25]:
def getSoupAtleti():
    base_url = "https://tokyo2020.coni.it/it/italia-team/atleti.html?cognome=&disciplina=0&regione=&provincia=&cerca=#"
    print(base_url)
    response = requests.get(base_url)
    html_page = response.text
    soup = bs(html_page)
    return soup

In [26]:
soup = getSoupAtleti()

https://tokyo2020.coni.it/it/italia-team/atleti.html?cognome=&disciplina=0&regione=&provincia=&cerca=#


In [27]:
nomi = [item.get_text().strip() for item in soup.select("span.nome")]
cognomi = [item.get_text().strip() for item in soup.select("span.cognome")]
discipline = [item.get_text().strip() for item in soup.select("span.disci")]
del discipline[276] #paltrinieri aveva due tipi di nuoto
diz = {'nome': nomi, 'cognome': cognomi, 'disciplina': discipline} 
temp2 = pd.DataFrame(diz)
temp2

,nome,cognome,disciplina
0,GIOVANNI,ABAGNALE,Canottaggio
1,VINCENZO,ABBAGNALE,Canottaggio
2,AHMED,ABDELWAHED,Atletica
3,DOMENICO,ACERENZA,Nuoto
4,VLADIMIR,ACETI,Atletica
...,...,...,...
379,FRANCESCO,ZAZA,Sport Equestri
380,LORENZO,ZAZZERI,Nuoto
381,SILVIA,ZENNARO,Vela
382,OSAMA,ZOGHLAMI,Atletica


Aggiungo secondo nome a BENEDICTA CHIGBOLU e sistemo accenti.

In [28]:
temp2["nome"].loc[87] = temp2["nome"].loc[87].replace("BENEDICTA", "MARIA BENEDICTA")

In [29]:
temp2["nome intero"] = temp2["nome"] + " " + temp2["cognome"]
temp2["nome intero"] = temp2["nome intero"].str.replace("A'", 'À')
temp2["nome intero"] = temp2["nome intero"].str.replace("E'", 'È')
temp2["nome intero"] = temp2["nome intero"].str.replace("I'", 'Ì')
temp2["nome intero"] = temp2["nome intero"].str.replace("O'", 'Ò')
temp2["nome intero"] = temp2["nome intero"].str.replace("U'", 'Ù')
temp2["nome"] = temp2["nome"].str.replace("A'", 'À')
temp2["nome"] = temp2["nome"].str.replace("E'", 'È')
temp2["nome"] = temp2["nome"].str.replace("I'", 'Ì')
temp2["nome"] = temp2["nome"].str.replace("O'", 'Ò')
temp2["nome"] = temp2["nome"].str.replace("U'", 'Ù')
temp2["cognome"] = temp2["cognome"].str.replace("A'", 'À')
temp2["cognome"] = temp2["cognome"].str.replace("E'", 'È')
temp2["cognome"] = temp2["cognome"].str.replace("I'", 'Ì')
temp2["cognome"] = temp2["cognome"].str.replace("O'", 'Ò')
temp2["cognome"] = temp2["cognome"].str.replace("U'", 'Ù')

temp2

,nome,cognome,disciplina,nome intero
0,GIOVANNI,ABAGNALE,Canottaggio,GIOVANNI ABAGNALE
1,VINCENZO,ABBAGNALE,Canottaggio,VINCENZO ABBAGNALE
2,AHMED,ABDELWAHED,Atletica,AHMED ABDELWAHED
3,DOMENICO,ACERENZA,Nuoto,DOMENICO ACERENZA
4,VLADIMIR,ACETI,Atletica,VLADIMIR ACETI
...,...,...,...,...
379,FRANCESCO,ZAZA,Sport Equestri,FRANCESCO ZAZA
380,LORENZO,ZAZZERI,Nuoto,LORENZO ZAZZERI
381,SILVIA,ZENNARO,Vela,SILVIA ZENNARO
382,OSAMA,ZOGHLAMI,Atletica,OSAMA ZOGHLAMI


In [30]:
temp2.to_csv("ListaAzzurri.csv", index=False)